In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<a href=\"https://colab.research.google.com/github/Epivitae/mProcess/blob/main/notebooks/advanced_usage.ipynb\">\n",
    "  <img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/>\n",
    "</a>\n",
    "\n",
    "# mProcess: Advanced Programmatic Usage\n",
    "\n",
    "**Author:** Dr. Kui Wang  \n",
    "**Version:** v4.1.0\n",
    "\n",
    "This notebook demonstrates how to use the `mProcess` core modules programmatically, bypassing the GUI. This is useful for:\n",
    "- Batch processing multiple files.\n",
    "- Integrating the analysis pipeline into other scripts.\n",
    "- Debugging specific modules.\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Environment Setup\n",
    "\n",
    "Since this notebook is located in the `notebooks/` directory, we need to add the parent directory (project root) to the system path to import the `core` package."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "import sys\n",
    "\n",
    "# Add project root to path\n",
    "project_root = os.path.abspath('..')\n",
    "if project_root not in sys.path:\n",
    "    sys.path.append(project_root)\n",
    "\n",
    "print(f\"✅ Project root added: {project_root}\")\n",
    "\n",
    "# Import core modules\n",
    "try:\n",
    "    from core import single_core\n",
    "    from core import dual_core_gui_adapter\n",
    "    from core._version import __version__\n",
    "    print(f\"✅ mProcess Core v{__version__} loaded successfully.\")\n",
    "except ImportError as e:\n",
    "    print(f\"❌ Error loading modules: {e}\")\n",
    "    print(\"Please ensure you are running this notebook from the 'notebooks' directory within the mProcess project.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Define Helper Functions\n",
    "\n",
    "The core functions accept an optional `on_step` callback, which is used by the GUI to update the progress bar. In a notebook environment, we can simply print a dot or a status message."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def simple_progress_callback():\n",
    "    \"\"\"Prints a dot to indicate progress.\"\"\"\n",
    "    print(\".\", end=\"\", flush=True)\n",
    "\n",
    "def verbose_progress_callback():\n",
    "    \"\"\"Just a placeholder if you want more verbose output.\"\"\"\n",
    "    pass"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Configuration\n",
    "\n",
    "Set the path to your raw Excel file here. \n",
    "> **Note:** Place your raw data file in the `notebooks` folder or provide an absolute path."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# REPLACE THIS with your actual file name\n",
    "FILE_PATH = \"example_data.xlsx\"\n",
    "\n",
    "if not os.path.exists(FILE_PATH):\n",
    "    print(f\"⚠️ Warning: File '{FILE_PATH}' not found. Please upload a file to test.\")\n",
    "else:\n",
    "    print(f\"📁 Ready to process: {FILE_PATH}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Run Single Channel Mode (Intensity)\n",
    "\n",
    "Use `single_core.run_main()` for standard intensity assays."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if os.path.exists(FILE_PATH):\n",
    "    print(\"🚀 Starting Single Channel Analysis...\")\n",
    "    \n",
    "    # Run the core logic\n",
    "    generated_files = single_core.run_main(FILE_PATH, on_step=simple_progress_callback)\n",
    "    \n",
    "    print(\"\\n\\n✅ Processing Complete!\")\n",
    "    print(\"Generated Files:\")\n",
    "    for f in generated_files:\n",
    "        print(f\" - {f}\")\n",
    "else:\n",
    "    print(\"❌ Skipped: No input file.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Run Dual Channel Mode (Ratio)\n",
    "\n",
    "Use `dual_core_gui_adapter.run_main()` for ratiometric assays (e.g., 485nm/420nm)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if os.path.exists(FILE_PATH):\n",
    "    print(\"🚀 Starting Dual Channel (Ratio) Analysis...\")\n",
    "    \n",
    "    # Run the core logic\n",
    "    # Note: This expects the Excel file to have the specific layout required for Ratio analysis\n",
    "    try:\n",
    "        generated_files = dual_core_gui_adapter.run_main(FILE_PATH, on_step=simple_progress_callback)\n",
    "        \n",
    "        print(\"\\n\\n✅ Processing Complete!\")\n",
    "        print(\"Generated Files:\")\n",
    "        for f in generated_files:\n",
    "            print(f\" - {f}\")\n",
    "    except Exception as e:\n",
    "        print(f\"\\n❌ Error: {e}\")\n",
    "        print(\"Make sure the input file has the correct format for Dual Channel analysis.\")\n",
    "else:\n",
    "    print(\"❌ Skipped: No input file.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Data Verification (Optional)\n",
    "\n",
    "You can use `pandas` to inspect the generated results immediately."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "\n",
    "# Example: Load the 'Kinetics.xlsx' generated by Single Channel mode\n",
    "output_file = \"Kinetics.xlsx\"\n",
    "\n",
    "if os.path.exists(output_file):\n",
    "    df = pd.read_excel(output_file)\n",
    "    print(\"📊 First 5 rows of result:\")\n",
    "    display(df.head())\n",
    "else:\n",
    "    print(f\"ℹ️ File {output_file} not found yet.\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}